<a href="https://colab.research.google.com/github/OlgaShevel/Others/blob/main/scruper_photo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Кастомный скрапер писался для выгрузки в colab фото с первой html-страницы Яндекс.картинки. Но может тянуть изображения и с некоторых других сайтов. Маркетплейсы и другие агрегаторы изображений ставят защиту или загружают фавиконы или угрозы.

**КАК УСТРОЕН**

Работает по поисковому запросу в строке ресурса. В примере ниже загружены фото по запросу на русском языке "автомобиль Lada". Можно писать на разных языках сколько угодно слов поиска (в разумных пределах, конечно).

С учетом запроса ресурс сгенерирует адресную строку, которую надо скопировать и вставить в код.

С первой html-страницы можно награбить в районе 200 фото. Если нобходимо грабить вторую, третью и т.д. страницы, достаточно внести соответствующие поправки руками в ссылку.

По ходу работы скрапер прорежает временную папку colab, удаляя некондционные файлы, и дает статистику по количеству оставшихся.

Через 3-4 запуска рекомендуется перегрузить скрапер - Яндекс тоже может выставить защиту.

**ЧТО ДЕЛАЕТ**

- тянет все форматы, которые сочтет изображением
- выбирает уникальные ссылки для скачивания
- игнорирует ошибки загрузки после первой попытки обращения к сайту хранения картинки
- игнорирует ссылки больше 230 символов (они, как правило, выдают ошибки при скачивании)
- раскладывает файлы по трем папкам в завсимости от размера (Big_over_2000px, Middle_500-2000px, Small_less_500px).
- В названиях файлов также указывается размер картинки и далее через нижнюю несколько первых символов адреса сайта (например, "2048x1536_avtomob.jpg").
- Папки с размерами зиппуются и автоматом скачиваются в папку загрузки компьютера (среда Windows, остальные операционки не пробовала).



In [1]:
!pip install transliterate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 855.4 kB/s eta 0:00:00


In [2]:
import requests
import re
import os
import zipfile
import random
import time
import shutil
import urllib.parse
import urllib.request
from transliterate import translit
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from PIL import Image

# Тут скачивается html-код сайта - soap-----------------------------------------
# Он дербанится на список ссылок - links

url = 'https://yandex.ru/images/search?text=%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%BE%D0%B1%D0%B8%D0%BB%D1%8C%20Lada' # запрос "автомобиль Lada"

parsed_url = urllib.parse.urlparse(url)
query_dict = urllib.parse.parse_qs(parsed_url.query)

# с get должно работать даже без ключа
text = urllib.parse.unquote(query_dict.get('text', [''])[0])
perem_rus = text.replace(' ', '_').title()
perem_rus = translit(text, 'ru', reversed=True)
perem_eng = perem_rus.replace(' ', '_').title()

if parsed_url.netloc != '':
  perem_rus = parsed_url.netloc.split('.')[0]
  perem_eng = translit(perem_rus, 'ru', reversed=True).replace(' ', '_').title()
else:
  query_dict = urllib.parse.parse_qs(parsed_url.query)
  text = urllib.parse.unquote(query_dict.get('text', [''])[0])
  perem_rus = text.replace(' ', '_').title()
  perem_eng = translit(perem_rus, 'ru', reversed=True).replace(' ', '_').title()

# Скачиваем html-код со ссылками -----------------------------------------------
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
#soup

spis_format = re.findall(r'[\'"](https[^\'" >]+\.(?:jpg|jpeg|gif|png))', str(soup))
unique_spis = set(spis_format)
formatted_spis = [f'{line}' for line in unique_spis]
links = formatted_spis
kol_all = len(links)

with open(f'Photo_{perem_eng}.txt', 'w') as f:
  f.write('\n'.join(links))

# Закачиваем ссылки img
# Тут ограничение по количеству попыток и времени коннекта с сервером-----------

import time
start_time = time.time()

# создаем папку и динамический path, чтобы он постоянно мог меняться------------
save_dir = f'Folder_{perem_eng}'
if not os.path.isdir(save_dir):
  os.makedirs(save_dir)
path = os.path.join('/content', save_dir)
if not os.path.exists(path):
    os.makedirs(path)

for i, ss in enumerate(links): # x это links
  ss = ss.strip()
  image_name = f'{ss.split("/")[-1]}'
  if len(image_name) <= 230:
    image_path = os.path.join(save_dir, image_name)
    # 1 - количество попыток соединения с сервером, если тот не отвечает
    for attempts in range(1):
      try:
        # 2 - количество секунд на каждую попытку
        response = requests.get(ss, timeout=2)
        with open(image_path, 'wb') as f:
          f.write(response.content)
        print(f'Успешно №{i+1} из {kol_all}: {image_name} from {ss}')
        #print(f'Успешно №{i+1}: {image_name} from {ss}')
        break # если загрузка прошла успешно, прерываем цикл попыток
      except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
        print(f'Ошибка загрузки картинки с {ss} на попытке {attempts+1}: {e}')
        time.sleep(10) # пауза перед следующей попыткой
  else:
    print(f'Ссылка {ss} больше 230 символов')

t = int(time.time() - start_time)
print('\nКачалось {:02d} часов {:02d} минут {:02d} секунд'.format(t//3600, (t%3600)//60, t%60))

print()
print(f'Количество ссылок: \nизначально - {len(spis_format)} \nуникальных - {kol_all}')
print()

num_files = len([file for file in os.listdir(path) if os.path.isfile(os.path.join(path, file))])
print(f'\nВ папке "{perem_eng}" лежит {num_files} файлов')

Успешно №1 из 181: lada-vesta-avtomobili-vaz-lada-vesta-avt-1528606.jpg from https://i.artfile.me/wallpaper/16-09-2020/1920x1080/lada-vesta-avtomobili-vaz-lada-vesta-avt-1528606.jpg
Ошибка загрузки картинки с https://rusautomobile.ru/wp-content/gallery/avtovaz-rassekretil-serijnuyu-lada-xray/13_compressed.jpg на попытке 1: HTTPSConnectionPool(host='rusautomobile.ru', port=443): Max retries exceeded with url: /wp-content/gallery/avtovaz-rassekretil-serijnuyu-lada-xray/13_compressed.jpg (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7912250be6e0>: Failed to resolve 'rusautomobile.ru' ([Errno -5] No address associated with hostname)"))
Ошибка загрузки картинки с https://zolya.ru/wp-content/uploads/9/9/b/99b74e3f763aa9e3668e711c4d5f6fa8.jpeg на попытке 1: HTTPSConnectionPool(host='zolya.ru', port=443): Max retries exceeded with url: /wp-content/uploads/9/9/b/99b74e3f763aa9e3668e711c4d5f6fa8.jpeg (Caused by NameResolutionError("<urllib3.connection.HTTPSConne

In [3]:
# ОБРАБОТКА ПАПКИ С ФАЙЛАМИ

# Проверяем, все ли файлы = img, переименовываем в соотв с размером ------------
files_list = []
for file in os.listdir(path):
  file_path = os.path.join(path, file)
  files_list.append(file_path)

i = 1
for file in files_list:
    try:
        with Image.open(file) as img:
            width, height = img.size
            filename, ext = os.path.splitext(os.path.basename(file))
            # тут я ограничиваю имя файла 20 символами после {width}x{height}_
            new_filename = f'{width}x{height}_{filename[:16-len(str(width))-len(str(height))-1].strip("_")}{ext}' if len(filename) > 20 else f'{width}x{height}_{filename}{ext}'
            new_file = os.path.join(path, new_filename)
            os.rename(file, new_file)
            files_list[i-1] = new_file  # Обновляем список файлов с новым именами
            #print(f'{new_filename}')
            i += 1
    except Exception as e:
        #print(f"WARNING: Файл {file} не является изображением или поврежден ({str(e)})")
        continue

# Удаляем некондиционные файлы -------------------------------------------------
num_deleted = 0

for file in os.listdir(path):
  if not re.match(r'^\d+x\d+_', file):
    file_path = os.path.join(path, file)
    os.remove(file_path)
    num_deleted += 1
    #print(f'Файл {file_path} удален')
print(f'\nУдалено {num_deleted} файлов')
print(f'\nВ папке "{perem_eng}" осталось {num_files - num_deleted} файлов')


# Копируем файлы в новую папку по условию --------------------------------------
new_path = os.path.join('/content', perem_eng)
paps = ['Small_less_500px', 'Middle_500-2000px', 'Big_over_2000px']
# создаем три подпапки в новой папке, куда копировать
for p in paps:
    os.makedirs(os.path.join(new_path, p), exist_ok=True)

for file in os.listdir(path):
    match = re.match(r'^(\d+)x(\d+)_.*$', file)
    if match:
        width = int(match.group(1))
        height = int(match.group(2))

        # переместить файл в нужную папку или удалить---------------------------
        if width > 4000 or height > 4000:
            os.remove(os.path.join(path, file))
        elif width < 500 or height < 500:
            shutil.copy(os.path.join(path, file), os.path.join(new_path, 'Small_less_500px'))
        elif width >= 500 and width <= 2000 and height >= 500 and height <= 2000:
            shutil.copy(os.path.join(path, file), os.path.join(new_path, 'Middle_500-2000px'))
        elif width > 2000 or height > 2000:
            shutil.copy(os.path.join(path, file), os.path.join(new_path, 'Big_over_2000px'))

# Зазиповать всю папку с подпапками---------------------------------------------
x = shutil.make_archive(new_path, 'zip', new_path)

size_bytes = os.path.getsize(f'{perem_eng}.zip')
size_mb = size_bytes / (1024 * 1024)

print(f'\nZip получен по пути {x}')
print(f"\nЗазипованный файл весит {size_mb:.2f} MB.")
print()

from google.colab import files
zip_path = os.path.join('/content', f'{perem_eng}.zip')
if os.path.exists(zip_path):
    files.download(zip_path)
print(f"Зазипованный файл скачался успешно")


Удалено 35 файлов

В папке "Yandex" осталось 122 файлов

Zip получен по пути /content/Yandex.zip

Зазипованный файл весит 51.46 MB.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Зазипованный файл скачался успешно
